# Part - 1: Generating h<sup>*</sup>(D) model

## Step - 1: Importing respective libraries and meta-dataset.csv

In [114]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, KFold, RepeatedStratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('./meta-dataset.csv')
df

,Unnamed: 0,beta_0,beta_1,beta_2,beta_3,beta_4,beta_5,beta_6,beta_7,label
0,0,0.116358,0.134907,0.278246,0.470489,0.0,0.0,0.0,0.0,1
1,1,0.096939,0.210459,0.269133,0.423469,0.0,0.0,0.0,0.0,1
2,2,0.067696,0.157957,0.483373,0.290974,0.0,0.0,0.0,0.0,1
3,3,0.085299,0.132486,0.328494,0.453721,0.0,0.0,0.0,0.0,1
4,4,0.026059,0.262215,0.288274,0.423453,0.0,0.0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...
221,221,0.000000,0.335975,0.664025,0.000000,0.0,0.0,0.0,0.0,1
222,222,0.000000,0.322368,0.677632,0.000000,0.0,0.0,0.0,0.0,1
223,223,0.062405,0.138508,0.512938,0.286149,0.0,0.0,0.0,0.0,1
224,224,0.001678,0.318792,0.677852,0.001678,0.0,0.0,0.0,0.0,1


In [3]:
features = df.iloc[:, 1:-1]
labels = df.iloc[:, [-1]]

In [4]:
features

,beta_0,beta_1,beta_2,beta_3,beta_4,beta_5,beta_6,beta_7
0,0.116358,0.134907,0.278246,0.470489,0.0,0.0,0.0,0.0
1,0.096939,0.210459,0.269133,0.423469,0.0,0.0,0.0,0.0
2,0.067696,0.157957,0.483373,0.290974,0.0,0.0,0.0,0.0
3,0.085299,0.132486,0.328494,0.453721,0.0,0.0,0.0,0.0
4,0.026059,0.262215,0.288274,0.423453,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
221,0.000000,0.335975,0.664025,0.000000,0.0,0.0,0.0,0.0
222,0.000000,0.322368,0.677632,0.000000,0.0,0.0,0.0,0.0
223,0.062405,0.138508,0.512938,0.286149,0.0,0.0,0.0,0.0
224,0.001678,0.318792,0.677852,0.001678,0.0,0.0,0.0,0.0


In [5]:
labels 

,label
0,1
1,1
2,1
3,1
4,1
...,...
221,1
222,1
223,1
224,1


## Step-2: Splitting the dataset to training data and testing data respectively

In [6]:
features_train, features_test, labels_train, labels_test = train_test_split(features, labels, test_size = 0.3)

print(f'features_train = \n{features_train}')
print(f'labels_train = \n{features_test}')
print(f'features_test = \n{labels_train}')
print(f'labels_test = \n{labels_test}')

features_train = 
       beta_0    beta_1    beta_2    beta_3    beta_4    beta_5    beta_6  \
147  0.032445  0.172654  0.000000  0.000000  0.040556  0.750869  0.003476   
152  0.051948  0.152597  0.000000  0.000000  0.048701  0.746753  0.000000   
136  0.017699  0.138643  0.502950  0.340708  0.000000  0.000000  0.000000   
115  0.047859  0.172544  0.000000  0.000000  0.080605  0.652393  0.046599   
121  0.101072  0.197550  0.289433  0.411945  0.000000  0.000000  0.000000   
..        ...       ...       ...       ...       ...       ...       ...   
46   0.000000  0.213904  0.401070  0.385027  0.000000  0.000000  0.000000   
16   0.000000  0.199787  0.523911  0.276302  0.000000  0.000000  0.000000   
191  0.000000  0.381022  0.467153  0.151825  0.000000  0.000000  0.000000   
204  0.118367  0.167347  0.289796  0.424490  0.000000  0.000000  0.000000   
168  0.000000  0.296296  0.635802  0.067901  0.000000  0.000000  0.000000   

     beta_7  
147     0.0  
152     0.0  
136     0.0  
1

## Step-3: Instantiating K-Fold Cross-Validation Algorithm

In [115]:
RepeatedStratifiedKFold = RepeatedStratifiedKFold(n_splits=5)
# kfold_cv_outer = KFold(n_splits=5, shuffle=True)

## Step-4: Performing Tradional Grid Search on both the Decision Tree model and the K-Nearest-Neighbour model
We perform grid first search on the decision tree model and the K-nearest-neighbour model seperately, as we first find the most optimal variation of each of the models seperately with its respective parameters and then the compare both. The best model is then chosen as our h<sup>*</sup>(D)

### Step-4.1: Initializing Decision-Tree model and selecting most relavant hyperparametes

The following hyperparameters were choosen to be used in the for the decision tree model:
1. Criterion: 
1. Splitter:
1. Max-Depth: The depth will detemine how well the model behaves on unseen data.
1. Min-Samples-Split: 
1. Class-weight: We have choosen the value of the class-weight parameter to be None because we assume there is no noise in the dataset. The reason for this assumption is because the it in mentioned in the project proposal that the data-preprpcessing stages for the pipeline have already been conducted for us.

In [121]:
decision_tree = DecisionTreeClassifier()
criterion_values = ('gini', 'entropy','log_loss')
splitter_values = ('best', 'random')
max_depth_values = range(50)
min_samples_split_values = (2,3,4,5,6,7,8,9)
decision_tree_parameters = {'criterion':criterion_values, 'splitter':splitter_values, 'max_depth':max_depth_values, 
                            'min_samples_split':min_samples_split_values, 'class_weight':['balanced']} # Need to verify if the class-weight parameter is important because there is no noise in the dataset


### Step-4.2: Performing Grid Search to find the optimal set of hyperparameters for the decision tree model

In [107]:
# accuracy_results = []
# params_results = []
# for training_idx, validate_idx in kfold_cv_outer.split(features_train):
#     features_training = features_train.iloc[training_idx]
#     features_validating =  features_train.iloc[validate_idx]
#     labels_training = labels_train.iloc[training_idx]
#     labels_validating = labels_train.iloc[validate_idx]
#     decision_tree_classifier = GridSearchCV(decision_tree, decision_tree_parameters, cv=RepeatedStratifiedKFold, refit=True)
#     decision_tree_classifier.fit(features_training, labels_training)
#     labels_valid_predict = decision_tree_classifier.predict(features_validating)
#     accuracy_results.append(accuracy_score(labels_validating, labels_valid_predict))
#     params_results.append(decision_tree_classifier.best_params_)
# print(accuracy_results)
# print(params_results)

/Users/shreyaskumar/miniforge3/envs/shreyas_ml/lib/python3.8/site-packages/sklearn/model_selection/_split.py:684: UserWarning: The least populated class in y has only 9 members, which is less than n_splits=10.
  warnings.warn(
/Users/shreyaskumar/miniforge3/envs/shreyas_ml/lib/python3.8/site-packages/sklearn/model_selection/_split.py:684: UserWarning: The least populated class in y has only 9 members, which is less than n_splits=10.
  warnings.warn(
/Users/shreyaskumar/miniforge3/envs/shreyas_ml/lib/python3.8/site-packages/sklearn/model_selection/_split.py:684: UserWarning: The least populated class in y has only 9 members, which is less than n_splits=10.
  warnings.warn(
/Users/shreyaskumar/miniforge3/envs/shreyas_ml/lib/python3.8/site-packages/sklearn/model_selection/_split.py:684: UserWarning: The least populated class in y has only 9 members, which is less than n_splits=10.
  warnings.warn(
/Users/shreyaskumar/miniforge3/envs/shreyas_ml/lib/python3.8/site-packages/sklearn/model_sel

[0.5625, 0.5625, 0.375, 0.4375, 0.4375, 0.4375, 0.625, 0.5, 0.6666666666666666, 0.4666666666666667]
[{'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 10, 'min_samples_split': 2, 'splitter': 'best'}, {'class_weight': 'balanced', 'criterion': 'log_loss', 'max_depth': 12, 'min_samples_split': 2, 'splitter': 'best'}, {'class_weight': 'balanced', 'criterion': 'entropy', 'max_depth': 12, 'min_samples_split': 2, 'splitter': 'best'}, {'class_weight': 'balanced', 'criterion': 'log_loss', 'max_depth': 10, 'min_samples_split': 2, 'splitter': 'best'}, {'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 12, 'min_samples_split': 2, 'splitter': 'random'}, {'class_weight': 'balanced', 'criterion': 'entropy', 'max_depth': 12, 'min_samples_split': 2, 'splitter': 'best'}, {'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 12, 'min_samples_split': 2, 'splitter': 'best'}, {'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 12, 'min_samples_split': 2, 'splitte

In [124]:
decision_tree_classifier = GridSearchCV(decision_tree, decision_tree_parameters, cv=RepeatedStratifiedKFold, refit=True)
decision_tree_classifier.fit(features_train, labels_train)

/Users/shreyaskumar/miniforge3/envs/shreyas_ml/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
2400 fits failed out of a total of 240000.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
2400 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/shreyaskumar/miniforge3/envs/shreyas_ml/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/shreyaskumar/miniforge3/envs/shreyas_ml/lib/python3.8/site-packages/sklearn/tree/_classes.py", line 969, in fit
    super().fit(
  File "/Users/shreyaskumar/miniforge3/envs/shreyas_ml/lib/python3.8/site-packages/sklearn/tre

GridSearchCV(cv=RepeatedStratifiedKFold(n_repeats=10, n_splits=5, random_state=None),
             estimator=DecisionTreeClassifier(),
             param_grid={'class_weight': ['balanced'],
                         'criterion': ('gini', 'entropy', 'log_loss'),
                         'max_depth': range(0, 100),
                         'min_samples_split': (2, 3, 4, 5, 6, 7, 8, 9),
                         'splitter': ('best', 'random')})

In [126]:
# decision_tree_classifier.score(features_test, labels_test)
labels_predict = decision_tree_classifier.predict(features_test)
print(accuracy_score(labels_test, labels_predict))
print("\n The best parameters across ALL searched params:\n",decision_tree_classifier.best_params_)

0.5735294117647058

 The best parameters across ALL searched params:
 {'class_weight': 'balanced', 'criterion': 'entropy', 'max_depth': 56, 'min_samples_split': 2, 'splitter': 'random'}


### Step-4.3: Performing Grid Search to find the optimal set of hyperparameters for the K-Nearest-Neighbour model

In [136]:
knn = KNeighborsClassifier()
knn_neighbours_range = range(4,113)
knn_parameters = {'n_neighbors': knn_neighbours_range, 'weights':('uniform', 'distance'), 'p':[2], # p=2 for euclidian
                    'metric':['minkowski']}

In [137]:
knn_classifier = GridSearchCV(knn, knn_parameters, cv=RepeatedStratifiedKFold, refit=True)
knn_classifier.fit(features_train, labels_train)

/Users/shreyaskumar/miniforge3/envs/shreyas_ml/lib/python3.8/site-packages/sklearn/neighbors/_classification.py:207: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/Users/shreyaskumar/miniforge3/envs/shreyas_ml/lib/python3.8/site-packages/sklearn/neighbors/_classification.py:207: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/Users/shreyaskumar/miniforge3/envs/shreyas_ml/lib/python3.8/site-packages/sklearn/neighbors/_classification.py:207: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/Users/shreyaskumar/miniforge3/envs/shreyas_ml/lib/python3.8/site-packages/sklearn/neighbors/_classification.

GridSearchCV(cv=RepeatedStratifiedKFold(n_repeats=10, n_splits=5, random_state=None),
             estimator=KNeighborsClassifier(),
             param_grid={'metric': ['minkowski'], 'n_neighbors': range(4, 113),
                         'p': [2], 'weights': ('uniform', 'distance')})

In [139]:
labels_predict = knn_classifier.predict(features_test)
print(accuracy_score(labels_test, labels_predict))
print("\n The best parameters across ALL searched params:\n",knn_classifier.best_params_)

0.6176470588235294

 The best parameters across ALL searched params:
 {'metric': 'minkowski', 'n_neighbors': 39, 'p': 2, 'weights': 'distance'}
